In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [8]:
def create_model(input_shape=(224, 224, 3)):

    base_model = tf.keras.applications.EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    
    # Fine-tune the last few layers
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

In [9]:
def prepare_data(data_dir, img_height=224, img_width=224, batch_size=32):

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True,
        seed=42
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        subset='validation',
        shuffle=True,
        seed=42
    )
    
    return train_generator, validation_generator

In [10]:
def train_model(model, train_generator, validation_generator, epochs=50):

    # Use cosine decay learning rate
    # initial_learning_rate = 0.001
    # decay_steps = epochs * len(train_generator)
    # lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    #     initial_learning_rate, decay_steps
    # )
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            # tf.keras.metrics.Precision(name='precision'),
            # tf.keras.metrics.Recall(name='recall'),
            # tf.keras.metrics.AUC(name='auc')
        ]
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.keras',
            monitor='val_auc',
            save_best_only=True,
            mode='max'
        )
    ]

    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs,
        callbacks=callbacks
    )

    return history

In [11]:
def plot_training_history(history):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'])
    
    # Loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'])
    
    # Precision & Recall
    ax3.plot(history.history['precision'])
    ax3.plot(history.history['recall'])
    ax3.set_title('Precision and Recall')
    ax3.set_ylabel('Score')
    ax3.set_xlabel('Epoch')
    ax3.legend(['Precision', 'Recall'])
    
    # AUC
    ax4.plot(history.history['auc'])
    ax4.plot(history.history['val_auc'])
    ax4.set_title('AUC')
    ax4.set_ylabel('AUC')
    ax4.set_xlabel('Epoch')
    ax4.legend(['Train', 'Validation'])
    
    plt.tight_layout()
    plt.show()

In [12]:
def main():
    # Set parameters
    IMG_HEIGHT = 224
    IMG_WIDTH = 224
    BATCH_SIZE = 32
    EPOCHS = 50
    
    DATA_DIR = 'augmented_dataset'
    
    print("GPU Available: ", tf.config.list_physical_devices('GPU'))

    train_generator, validation_generator = prepare_data(
        DATA_DIR,
        IMG_HEIGHT,
        IMG_WIDTH,
        BATCH_SIZE
    )
    
    model = create_model((IMG_HEIGHT, IMG_WIDTH, 3))
    model.summary()
    
    history = train_model(model, train_generator, validation_generator, EPOCHS)
    
    plot_training_history(history)
    
    model.save('final_oral_cancer_model.keras')

if __name__ == "__main__":
    main()

GPU Available:  []
Found 3199 images belonging to 2 classes.
Found 799 images belonging to 2 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 7, 7, 1536)     │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,705,136 (44.65 MB)

 Trainable params: 4,293,217 (16.38 MB)

...

c:\Users\EliteBook\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5194 - auc: 0.5307 - loss: 0.9530 - precision: 0.5223 - recall: 0.5247

c:\Users\EliteBook\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 373s 3s/step - accuracy: 0.5193 - auc: 0.5305 - loss: 0.9525 - precision: 0.5222 - recall: 0.5246 - val_accuracy: 0.5006 - val_auc: 0.6912 - val_loss: 0.6937 - val_precision: 0.5006 - val_recall: 1.0000 - learning_rate: 9.9901e-04
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 282s 3s/step - accuracy: 0.4967 - auc: 0.5048 - loss: 0.7975 - precision: 0.5043 - recall: 0.5180 - val_accuracy: 0.5745 - val_auc: 0.7115 - val_loss: 0.6596 - val_precision: 0.8191 - val_recall: 0.1925 - learning_rate: 9.9606e-04
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 296s 3s/step - accuracy: 0.5266 - auc: 0.5320 - loss: 0.7543 - precision: 0.5285 - recall: 0.5416 - val_accuracy: 0.6345 - val_auc: 0.7178 - val_loss: 0.6614 - val_precision: 0.7621 - val_recall: 0.3925 - learning_rate: 9.9114e-04
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 301s 3s/step - accuracy: 0.4994 - auc: 0.5037 - loss: 0.7666 - precision: 0.5048 - recall: 0.4925 - val_accuracy: 0.6646 - val_auc: 0.6333 - val_loss: 0.6326 

TypeError: This optimizer was created with a `LearningRateSchedule` object as its `learning_rate` constructor argument, hence its learning rate is not settable. If you need the learning rate to be settable, you should instantiate the optimizer with a float `learning_rate` argument.